## Score Bootstrap based on Kline and Santos (2012)

In [1]:
import numpy as np
from wildboottest.wildboottest import wildboottest
from wildboottest.weights import draw_weights
from statsmodels.discrete.discrete_model import Probit
from numba import prange, jit
import statsmodels.api as sm
import pandas as pd
import stata_setup
stata_setup.config(path = "/Applications/Stata", 
                   edition = "mp")
from pystata import stata
import sfi
from joblib import Parallel, delayed


  ___  ____  ____  ____  ____ ®
 /__    /   ____/   /   ____/      Stata 18.0
___/   /   /___/   /   /___/       MP—Parallel Edition

 Statistics and Data Science       Copyright 1985-2023 StataCorp LLC
                                   StataCorp
                                   4905 Lakeway Drive
                                   College Station, Texas 77845 USA
                                   800-782-8272        https://www.stata.com
                                   979-696-4600        service@stata.com

Stata license: Single-user 2-core  perpetual
Serial number: 501806367191
  Licensed to: Aleksandr Michuda
               Cornell University

Notes:
      1. Unicode is supported; see help unicode_advice.
      2. More than 2 billion observations are allowed; see help obs_advice.
      3. Maximum number of variables is set to 5,000 but can be increased;
          see help set_maxvar.


## Stata Code from Paper

In [2]:
%%stata

****************************************************
*  Code for Table 2 of Kline and Santos (2011)     *
*  This file computes Analytical Wald and LM tests *
*  along with Score bootstrapped tests.   	   *
*  Some other results not reported in the paper    *
*  are also included.				   *	
****************************************************

version 11.1
set seed 12345

**mata subroutine to compute recentered clustered outer products**
**inputs a matrix S of scores and a cluster summing matrix L**
**outputs clustered outer product matrix**
**note this program assumes a balanced design**
**and proper sort order**
cap mata: mata drop clustOPG()
mata: 
real matrix clustOPG(real matrix S,real matrix L){
real scalar N
real matrix Scent, Scentsum, OPGcent

N=rows(S)
Scent=S - J(N,1,1)*mean(S)
Scentsum=L'*Scent
OPGcent=Scentsum'*Scentsum
return(OPGcent)
}
end



/* cap prog drop sim_probit
prog def sim_probit, rclass
syntax [, c(integer 5) m(integer 50) f(integer 0) r(integer 199)] */
*Notes on syntax: 
*c - # of clusters
*m - # of obs per cluster
*f - controls whether to use mixture regressor
*r - # of bootstrap reps per simulation

local c 10
local m 100
local f 0
local r 1000

**Generate data**
drop _all
set obs `c'  //generate c clusters

gen x=invnorm(uniform()) //covariate
gen mix=uniform()>.1 //contamination probability

if `f'==1{
	gen D=(mix*invnorm(uniform()) + (1-mix)*(2+3*invnorm(uniform())))/4 //regressor of interest
}
else{
	gen D=(invnorm(uniform()))/4 //regressor of interest
}

gen v=invnorm(uniform())


gen c=_n	//generate cluster id

expand `m'	//make m obs per cluster

replace x=(x+invnorm(uniform()))/4  //add some within cluster variation in x

gen y=x+D+v/sqrt(2)+invnorm(uniform())/sqrt(2)  //form outcome
replace y=y>0
sort c

**Restricted**
constraint 1 D=1
glm y x D, constraint(1) fam(bin) link(probit)
predict pr
predict xbr, xb
gen phir=normalden(xbr)
gen er=(y-pr)*phir/(pr*(1-pr)) //quasi-residual
gen wr=phir^2/(pr*(1-pr)) //weights for hessian
gen one=1

mat accum Hr=one x D [iw=wr], nocons //form hessian */


**Unrestricted**
probit y x D, cluster(c)
global b=_b[D]
local stdof=sqrt((`m'*`c'-3)/(`m'*`c'-1)) //inverse stata DOF correction
global W=((_b[D]-1)/(_se[D]*`stdof'))^2 //save analytical Wald

predict pu
predict xbu, xb
gen phiu=normalden(xbu)
gen eu=(y-pu)*phiu/(pu*(1-pu)) //quasi-residual
gen wu=phiu^2/(pu*(1-pu)) //weights for hessian

local dfk=(`c'/(`c'-1))
mata: L=I(`c')#J(`m',1,1) //matrix for summing across clusters
mata: X=st_data(.,("one", "x", "D"))
mata: C=(0\0\1)		//constraint matrix -- third coefficient is restricted

mata: Hr=st_matrix("Hr")
mata: Hrinv=invsym(Hr)
mata: Ar=C'*Hrinv		//form A_n


**Prepare influence function wald
mat accum Hu=one x D [iw=wu], nocons //form hessian
mata: Hu=st_matrix("Hu")
mata: Huinv=invsym(Hu)
mata: Au=C'*Huinv
mata: eu=st_data(.,"eu")
mata: OPGucent=clustOPG(X:*eu,L)
mata: Vinvu=invsym(Au*OPGucent*`dfk'*Au') //compute inverse of variance of influence function


save "sim_sample.dta", replace

mata: S = X:*eu

**Bootstrap**
gen u=.
gen ystar=.
gen ernew=.
gen eunew=.
gen pos=.


mat Ts=J(1,6,.) //store bootstrap statistics

by c: replace u=uniform()
by c: replace pos=u[1]<.5  //cluster level rademacher indicator
gen radem = 2*pos - 1

qui replace ernew=(2*pos-1)*er  //weighted residual
qui replace eunew=(2*pos-1)*eu  //weighted residual

*Score bootstrap LM -- Rademacher
mata: er=st_data(.,"ernew")
mata: Sr=colsum(X:*er)
mata: OPGrcent=clustOPG(X:*er,L)
mata: lm=Sr*Ar'*invsym(Ar*(OPGrcent)*`dfk'*Ar')*Ar*Sr'
mata: st_numscalar("lm",lm)
mat Ts[1,1]=lm	     //save bootstrap LM 


*Score Wald -- Rademacher
mata: eu=st_data(.,"eunew")
mata: Su=colsum(X:*eu)
mata: OPGucent=clustOPG(X:*eu,L)
mata: wald=Su*Au'*invsym(Au*OPGucent*`dfk'*Au')*Au*Su'
mata: st_numscalar("wald",wald)
mat Ts[1,2]=wald	     //save bootstrap Wald

*Score bootstrap Wald combining restricted and unrestricted scores -- Rademacher
mata: wald_r=Sr*Ar'*invsym(Au*OPGucent*`dfk'*Au')*Ar*Sr'
mata: st_numscalar("wald_r",wald)
mat Ts[1,3]=wald	     //save bootstrap Wald





. 
. ****************************************************
. *  Code for Table 2 of Kline and Santos (2011)     *
. *  This file computes Analytical Wald and LM tests *
. *  along with Score bootstrapped tests.            *
. *  Some other results not reported in the paper    *
. *  are also included.                              *    
. ****************************************************
. 
. version 11.1

. set seed 12345

. 
. **mata subroutine to compute recentered clustered outer products**
. **inputs a matrix S of scores and a cluster summing matrix L**
. **outputs clustered outer product matrix**
. **note this program assumes a balanced design**
. **and proper sort order**
. cap mata: mata drop clustOPG()

. mata: 
------------------------------------------------- mata (type end to exit) -----
: real matrix clustOPG(real matrix S,real matrix L){
> real scalar N
> real matrix Scent, Scentsum, OPGcent
> 
> N=rows(S)
> Scent=S - J(N,1,1)*mean(S)
> Scentsum=L'*Scent
> OPGcent=Scent

In [3]:
N = 1000
rng = np.random.default_rng(seed=1)
num_X = 2


# generates fake probit data for testing (Not used in favor of data generated from paper)
def probit_data(N, num_X):
    # generate X and error
    epsilon = rng.normal(size=N)
    X = sm.add_constant(rng.normal(size=(N, num_X)))
    beta = np.arange(1,num_X+2)
    
    true_y = X @ beta + epsilon
    
    y_star = np.where(true_y >0, 1,0)
    
    return y_star, true_y, X

y_star, true_y, X = probit_data(1000, 3)

clusters = rng.choice(range(100), size=1000)

df = stata.pdataframe_from_data()

p = Probit.from_formula("y ~ x + D", data=df)

In [4]:
p.fit(cov_type='cluster', cov_kwds={'groups' : df['c']}).summary()

Optimization terminated successfully.
         Current function value: 0.665335
         Iterations 4


<class 'statsmodels.iolib.summary.Summary'>
"""
                          Probit Regression Results                           
==============================================================================
Dep. Variable:                      y   No. Observations:                 1000
Model:                         Probit   Df Residuals:                      997
Method:                           MLE   Df Model:                            2
Date:                Thu, 06 Jun 2024   Pseudo R-squ.:                 0.04010
Time:                        19:23:11   Log-Likelihood:                -665.33
converged:                       True   LL-Null:                       -693.13
Covariance Type:              cluster   LLR p-value:                 8.494e-13
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.0358      0.308      0.116      0.908      -0.568       0.640
x              1.0579      0.396      2.670      0.008       0.281       1.835
D              0.0943      1.043      0.090      0.928      -1.950       2.139
==============================================================================
"""

In [5]:
L = np.array(sfi.Mata.get("L"))
S = np.array(sfi.Mata.get("S"))
OPGucent = np.array(sfi.Mata.get("OPGucent"))

In [6]:
p.fit().tvalues

Optimization terminated successfully.
         Current function value: 0.665335
         Iterations 4


Intercept    0.856059
x            7.202267
D            0.662808
dtype: float64

In [16]:
from typing import Union, TypeVar
import numpy.typing as npt
from statsmodels.base.model import Model

ArrayLike = Union[npt.ArrayLike, pd.DataFrame, pd.Series]

class IncorrectModelException(Exception):
    pass

class ScoreWildBootstrap:
    
    def __init__(self,
                 mod: Model, 
                 clusters=Union[None, ArrayLike]) -> None:
        """Score wild bootstrap as in Kline and Santos (2012). Perturbs the score of using wild bootstrap weights and calculates wald statistics.
        Optionally with restricted score or unrestricted scores.

        Args:
            mod (Model): A statsmodels model that has a `score_obs` and `hessian` method.
            clusters (ArrayLike|None, optional): A dataframe or array of shape Nx1 that gives the cluster membership of each of the N observations. Defaults to Union[None, ArrayLike].
        """        
        
        if not hasattr(mod, "score_obs"):
            raise IncorrectModelException(f"{mod} does not implement `score_obs`.")
        if not hasattr(mod, "hessian"):
            raise IncorrectModelException("{mod} does not implement `hessian`.")
        
        self.mod = mod
        
        self.fitted_mod = self.mod.fit(cov_type='cluster', cov_kwds = {'groups' : clusters})
        
        self.beta_hat = self.fitted_mod.params
        
        if clusters is not None:
            self.cluster_ids, self.cluster_obs = np.unique(clusters, return_counts=True)
            self.clusters = clusters
            self.num_clusters = len(self.cluster_ids)
        else:
            self.clusters = np.ones(self.mod.endog.shape[0])
            self.cluster_ids = np.array([1])
            self.num_clusters = 1
            
    def get_analytical_wald(self) -> pd.Series:
        """Calculates cluster-robust analytical wald test for each parameter.
        """        
        return self.fitted_mod.tvalues**2

    def wald_test(self, W: ArrayLike, restricted_residuals: bool =False, 
                  R: ArrayLike =None, r: Union[None, int, float]=None) -> Union[ArrayLike, float]:
        """Calculates Wald statistic, given perturbation weights, scores and hessians from the model.

        Args:
            W (ArrayLike): Weight vector used to perturb observation score
            restricted_residuals (bool, optional): Whether to construct the test with the restricted score. Defaults to False.
                \begin{aligned}
                    & T_{n, 1}^{\star s} \equiv\left(R H_n^{-1} S_n^{\star}\left(\hat{\beta}_u\right)\right)^{\prime}\left(R H_n^{-1} \sum_n^{\star s}\left(\hat{\beta}_u\right) H_n^{-1} R^{\prime}\right)^{-1}\left(R H_n^{-1} S_n^{\star}\left(\hat{\beta}_u\right)\right) \\
                    & T_{n, 2}^{\star s} \equiv\left(R H_n^{-1} S_n^{\star}\left(\hat{\beta}_r\right)\right)^{\prime}\left(R H_n^{-1} \sum_n^{\star s}\left(\hat{\beta}_u\right) H_n^{-1} R^{\prime}\right)^{-1}\left(R H_n^{-1} S_n^{\star}\left(\hat{\beta}_r\right)\right)
                \end{aligned}
                See Equations 50 and 51 from Kline and Santos (2012).
            R (ArrayLike, optional): Restriction matrix for test. Defaults to None.
            r (Union[None, int, float], optional): test restriction. Defaults to None.

        Returns:
            Union[ArrayLike, float]: A Wald Statistic based on the perturbed weights.
        """        
        
                        
        hessian_inv = -np.linalg.inv(self.mod.hessian(self.beta_hat))
        
        if restricted_residuals: # if we use the restricted score in the test
            if r is None:
                raise ValueError("Must specify coefficient constraint to restrict residuals")
            score_beta = self.beta_hat.copy()
            score_beta[np.where(R==1)[0]] = 1 
        else:
            score_beta = self.beta_hat
        
        # make array of weights divided by cluster membership
        W_Jn = np.select([self.clusters == c for c in self.cluster_ids], W.flatten())[:,np.newaxis]
        # divide weights by observations per clusters            
        perturbed_score_obs = self.mod.score_obs(self.beta_hat) * W_Jn
        
        # multiply by cluster membership
        L = pd.get_dummies(self.clusters).astype(int).values
        
        perturbed_score_obs_mean = (((L/(L.sum(axis=0)-1)).T @ perturbed_score_obs)).mean(axis=0)
        
        demeaned_perturbed_score_obs = perturbed_score_obs - perturbed_score_obs_mean 
        
        demeaned_perturbed_score_obs_L = L.T @ demeaned_perturbed_score_obs
        
        perturbed_score = (self.mod.score_obs(score_beta) * W_Jn).sum(axis=0)
        
        # equation 50 from the paper
        bread = R @ hessian_inv @ perturbed_score
        
        opg_centered = demeaned_perturbed_score_obs_L.T @ demeaned_perturbed_score_obs_L
        
        meat = R @ hessian_inv @ opg_centered @ hessian_inv @ R.T
        
        if isinstance(bread, np.ndarray):
            meat_inv = np.linalg.inv(meat)
            return bread.T @ meat_inv @ bread
        else:
            return (bread)**2/meat 
        
    def bootstrap(self, R: ArrayLike, 
                  W: Union[ArrayLike,None]=None, 
                  r: Union[int, float, None]=None, 
                  n_jobs: int = -1, 
                  rng: Union[np.random.Generator, None] = None, 
                  seed: Union[int, None] = None, 
                  bootstrap_num: int =100, 
                  restricted_residuals: bool=False, 
                  full_enumeration: bool=False) -> ArrayLike:
        """Bootstraps Wald Statistics by generating weights and continuously perturbing the score

        Args:
            R (ArrayLike, optional): Restriction Matrix. Defaults to None.
            r (Union[int, float], optional): restriction. Defaults to None.
            W (ArrayLike, optional): Weights vector. If not specified, Rademacher weights will be generated. Defaults to None.
            n_jobs (int, optional): Number of jobs to dispath for parallel processing. Defaults to -1, or all cores.
            rng (Union[np.random.Generator, None], optional): random generator that can be used for reproducibility. Defaults to None.
            seed (Union[int, None], optional): seed for random generator. Defaults to None.
            bootstrap_num (int, optional): number of bootstraps to perform. Defaults to 100.
            restricted_residuals (bool, optional): Whether to use restricted score in constructing Wald. Defaults to False.
            full_enumeration (bool, optional): Whether to generate the full set of random weight possibilities. Defaults to False.

        Returns:
            ArrayLike: A `bootstrap_num` x 1 matrix.
        """        
        
        def bootstrapper(restricted_residuals, R, r, full_enumeration, rng, W):
            if rng is None:
                rng=np.random.RandomState(seed)

            if W is None:
                W = draw_weights('rademacher',
                        full_enumeration=full_enumeration,
                        N_G_bootcluster=self.num_clusters,
                        boot_iter=1,
                        rng=rng)[0]
            
            return self.wald_test(W = W, restricted_residuals=restricted_residuals, R=R, r=r)
        
        walds = Parallel(n_jobs=n_jobs, backend='loky')(delayed(bootstrapper)(restricted_residuals=restricted_residuals, 
                                                             R=R, r=r, full_enumeration=full_enumeration, rng=rng, W=W) for b in range(bootstrap_num))
        
        return np.array(walds)
    
    
    def get_wald_boot(self, n_jobs: int =-1, 
                      bootstrap_num: int =100, 
                      restricted_residuals: bool=False, 
                      full_enumeration: bool=False, 
                      rng: Union[np.random.Generator, None] =None, 
                      seed: Union[int, None]=None,
                      W= Union[ArrayLike,None]) -> ArrayLike:
        """Generates Wald Statistics for each parameter separately.

        Args:
            n_jobs (int): Number of jobs to dispath for parallel processing. Defaults to -1, or all cores.
            bootstrap_num (int, optional): number of bootstraps to perform. Defaults to 100.
            restricted_residuals (bool, optional): Whether to use restricted score in constructing Wald. Defaults to False.
            full_enumeration (bool, optional): Whether to generate the full set of random weight possibilities. Defaults to False.
            rng (Union[np.random.Generator, None], optional): random generator that can be used for reproducibility. Defaults to None.
            seed (Union[int, None], optional): seed for random generator. Defaults to None.
            W ( Union[ArrayLike,None], optional): Weights vector. If not specified, Rademacher weights will be generated. Defaults to None.

        Returns:
            ArrayLike: A `bootstrap_num` x k matrix where k is the number of parameters.
        """        
        
        # generate restriction vectors for each variable for beta=0
        R = np.identity(self.beta_hat.shape[0])
                
        wald_boot = np.zeros(shape=(bootstrap_num, self.beta_hat.shape[0]))
        
        for i, R_vec in enumerate(R):
            wald_boot[:, i] = self.bootstrap(R = R_vec, r = 0, n_jobs=n_jobs,
                                          rng=rng, seed=seed, bootstrap_num=bootstrap_num,
                                          restricted_residuals=restricted_residuals,
                                          full_enumeration=full_enumeration,
                                          W=None)
            
        return wald_boot
        
        
    def get_pvalue(self, pval_type: str='two-tailed', **kwargs) -> ArrayLike:
        """Generates p-values by comparing analytical wald statistics with boostrapped, perturbed Wald statistics

        Args:
            pval_type (str, optional): The type of p-value to generate. Defaults to 'two-tailed'.
            kwargs can are used for specifying other options for `get_wald_boot`.

        Returns:
            ArrayLike: An array of p-values for each parameter
        """        
        
        wald_boot = self.get_wald_boot(**kwargs)
        
        analytical_wald = self.get_analytical_wald()
        
        if pval_type == "two-tailed":
            pvalue = np.where(wald_boot > analytical_wald.values, 1, 0).mean(axis=0)
        elif pval_type == "equal-tailed":
            pl = np.where(analytical_wald.values < wald_boot, 1,0).mean(axis=0)
            ph = np.where(analytical_wald.values > wald_boot, 1,0).mean(axis=0)
            pvalue = 2 * min(pl, ph)
        elif pval_type == ">":
            pvalue = np.where(analytical_wald.values < wald_boot,1,0).mean(axis=0)
        else:
            pvalue = np.where(analytical_wald.values > wald_boot,1,0).mean(axis=0)
            
        return pvalue
    

In [17]:
sb = ScoreWildBootstrap(p, clusters=df['c'])

ts = sb.get_pvalue(n_jobs=1, bootstrap_num=1000)

ts

Optimization terminated successfully.
         Current function value: 0.665335
         Iterations 4


array([0.909, 0.028, 0.931])

In [18]:
def score_obs(self, params):
    X= self.exog
    y = self.endog
    return (X.T*(y - X @ params)).T



In [19]:
sm.OLS.score_obs = score_obs

In [20]:
# linear_mod = sm.OLS(true_y, np.column_stack([X, rng.normal(size=1000)]))

linear_mod = sm.OLS.from_formula("y ~ x +D", data=df)

In [21]:
wildboottest(linear_mod, B=1000, weights_type='rademacher',
                impose_null=True, bootstrap_type='11')

| param     |   statistic |   p-value |
|:----------|------------:|----------:|
| Intercept |      32.953 |     0.000 |
| x         |       7.820 |     0.000 |
| D         |       0.632 |     0.526 |


/Users/lordflaron/Documents/wildboottest/.venv/lib/python3.9/site-packages/tabulate/__init__.py:107: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  (len(row) >= 1 and row[0] == SEPARATING_LINE)
/Users/lordflaron/Documents/wildboottest/.venv/lib/python3.9/site-packages/tabulate/__init__.py:108: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  or (len(row) >= 2 and row[1] == SEPARATING_LINE)


,statistic,p-value
param,,
Intercept,[32.95279531027586],0.000
x,[7.820060460734793],0.000
D,[0.6320615764783465],0.526


In [22]:
sb = ScoreWildBootstrap(linear_mod, clusters=df['c'])

ts = sb.get_pvalue(n_jobs=1, bootstrap_num=1000)

ts

array([0.   , 0.001, 0.868])